In [1]:
import pandas as pd
import numpy as np
from tflearn.data_utils import build_hdf5_image_dataset
import h5py
from tflearn.data_utils import shuffle
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
import tflearn

Building the train dataset

In [4]:
####Build the dataset (only required once)
id_train = pd.read_csv('../Data/id_train.csv')#replace by path of the id_train file 
file = pd.DataFrame(id_train['Id'].apply(lambda x: './train_imgs/' + str(x) + '.jpg' ))#Build the path to each image
file['label'] = id_train['label'] - 1 #Label goes from 1 to 4, we want 0 to 3 
file.to_csv('file_train.csv', sep = ' ', header = False, index=False)
dataset_file = 'file_train.csv'
build_hdf5_image_dataset(dataset_file, image_shape=(128, 128), mode='file', output_path='dataset.h5', categorical_labels=True, normalize=True)

,Id,label
0,./train_imgs/-3935637.jpg,3
1,./train_imgs/-2815459.jpg,2
2,./train_imgs/-2355684.jpg,2
3,./train_imgs/-2343303.jpg,3
4,./train_imgs/-1981985.jpg,1
5,./train_imgs/-1964319.jpg,2
6,./train_imgs/-1937877.jpg,2
7,./train_imgs/-1813560.jpg,2
8,./train_imgs/-1586982.jpg,3
9,./train_imgs/-1582718.jpg,2


Loading the Dataset

In [2]:
#Loading the dataset
h5f = h5py.File('train_dataset.h5', 'r')
X  = h5f['X']
Y = h5f['Y']

In [3]:
# Shuffle the data
X, Y = shuffle(X, Y)

In [4]:
X.shape

(8000, 128, 128, 3)

In [5]:
Y.shape

(8000, 4)

In [6]:
# Make sure the data is normalized
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center(mean =  0.457435)
img_prep.add_featurewise_stdnorm(std = 0.233162)

In [7]:
# Create extra synthetic training data by flipping, rotating and blurring the images on our data set.
img_aug = ImageAugmentation()
#img_aug.add_random_flip_leftright() #Flipping will make 
img_aug.add_random_rotation(max_angle=25.)
img_aug.add_random_blur(sigma_max=3.)

Building the network

In [8]:
# Input is a 128x128 image with 3 color channels (red, green and blue)
network = input_data(shape=[None, 128, 128, 3],
                     data_preprocessing=img_prep,
data_augmentation=img_aug)

In [9]:
# Step 1: Convolution
network = conv_2d(network, 128, 3, activation='relu')

In [10]:
# Step 2: Max pooling
network = max_pool_2d(network, 2)

In [11]:
# Step 3: Convolution again
network = conv_2d(network, 256, 3, activation='relu')

In [12]:
# Step 4: Convolution yet again
network = conv_2d(network, 256, 3, activation='relu')

In [13]:
# Step 5: Max pooling again
network = max_pool_2d(network, 2)

In [14]:
# Step 6: Fully-connected 512 node neural network
network = fully_connected(network, 512, activation='relu')

In [15]:
# Step 7: Dropout - throw away some data randomly during training to prevent over-fitting
network = dropout(network, 0.5)

In [16]:
# Step 8: Fully-connected neural network with 4 outputs to make the final prediction
network = fully_connected(network, 4, activation='softmax')

In [17]:
# Tell tflearn how we want to train the network
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy', n_classes = 4,learning_rate=0.01)

In [18]:
# Wrap the network in a model object
model = tflearn.DNN(network, tensorboard_verbose=0)

Training the model

In [19]:
epochs = 50

In [ ]:
model.fit(X, Y, n_epoch=50, shuffle=True,
          show_metric=True, batch_size=96,
          snapshot_epoch=True,
run_id='roof-classifier', validation_set=0.2)
model.save("roof-classifier.tfl")
print("Network trained and saved as roof-classifier.tfl!")

Training Step: 4  | total loss: 15.22782
| Adam | epoch: 000 | loss: 15.22782 - acc: 0.3070 -- iter: 0384/6400


Prediction